# Hydra Routing Debug Notebook

Tests every layer of the routing stack to identify issues.

## Routing Layers:
1. DNS Resolution
2. Traefik Entrypoint (port 80)
3. Traefik Router Matching
4. Traefik Middlewares (forwardAuth, stripPrefix)
5. Traefik Service Backend
6. Docker Network Connectivity
7. Container Service Response

In [ ]:
import requests
import socket
import json
import subprocess

TRAEFIK_API = 'http://localhost:8081/api'
HYDRA_HOST = 'hydra.local'
TEST_USER = 'user1'

## Layer 1: DNS Resolution

In [ ]:
def test_dns():
    try:
        ip = socket.gethostbyname(HYDRA_HOST)
        print(f'[OK] DNS: {HYDRA_HOST} -> {ip}')
        return ip == '127.0.0.1'
    except socket.gaierror as e:
        print(f'[FAIL] DNS: {e}')
        print('  Add "127.0.0.1 hydra.local" to /etc/hosts')
        return False

test_dns()

## Layer 2: Traefik Entrypoint

In [ ]:
def test_traefik_entrypoint():
    try:
        r = requests.get(f'{TRAEFIK_API}/entrypoints', timeout=5)
        entrypoints = r.json()
        print('Entrypoints:')
        for ep in entrypoints:
            print(f"  {ep['name']}: {ep.get('address', 'N/A')}")
        
        web_ep = next((e for e in entrypoints if e['name'] == 'web'), None)
        if web_ep:
            print(f'[OK] Web entrypoint: {web_ep.get("address")}')
            return True
        print('[FAIL] Web entrypoint not found')
        return False
    except Exception as e:
        print(f'[FAIL] Traefik API: {e}')
        return False

test_traefik_entrypoint()

## Layer 3: Traefik Router Matching

In [ ]:
def test_traefik_routers():
    try:
        r = requests.get(f'{TRAEFIK_API}/http/routers', timeout=5)
        routers = r.json()
        
        print('HTTP Routers:')
        print('-' * 50)
        for router in routers:
            status = '[OK]' if router.get('status') == 'enabled' else '[--]'
            print(f"{status} {router['name']}")
            print(f"     Rule: {router.get('rule', 'N/A')}")
            print(f"     Service: {router.get('service', 'N/A')}")
            print(f"     Middlewares: {router.get('middlewares', [])}")
        
        student_routers = [r for r in routers if 'student' in r['name'].lower()]
        if student_routers:
            print(f'\n[OK] {len(student_routers)} student router(s)')
            return True
        print('\n[FAIL] No student routers found')
        return False
    except Exception as e:
        print(f'[FAIL] {e}')
        return False

test_traefik_routers()

## Layer 4: Traefik Middlewares

In [ ]:
def test_traefik_middlewares():
    try:
        r = requests.get(f'{TRAEFIK_API}/http/middlewares', timeout=5)
        middlewares = r.json()
        
        print('HTTP Middlewares:')
        print('-' * 50)
        for mw in middlewares:
            status = '[OK]' if mw.get('status') == 'enabled' else '[--]'
            print(f"{status} {mw['name']}")
            if 'forwardAuth' in mw:
                print(f"     forwardAuth -> {mw['forwardAuth'].get('address', 'N/A')}")
            elif 'stripPrefix' in mw:
                print(f"     stripPrefix: {mw['stripPrefix'].get('prefixes', [])}")
        
        auth_mw = [m for m in middlewares if 'auth' in m['name'].lower()]
        strip_mw = [m for m in middlewares if 'strip' in m['name'].lower()]
        
        if not auth_mw or not strip_mw:
            print('\n[FAIL] Missing middlewares')
            return False
        print(f'\n[OK] {len(auth_mw)} auth + {len(strip_mw)} strip middlewares')
        return True
    except Exception as e:
        print(f'[FAIL] {e}')
        return False

test_traefik_middlewares()

## Layer 5: Traefik Services (Backend Resolution)

In [ ]:
def test_traefik_services():
    try:
        r = requests.get(f'{TRAEFIK_API}/http/services', timeout=5)
        services = r.json()
        
        print('HTTP Services:')
        print('-' * 50)
        for svc in services:
            status = '[OK]' if svc.get('status') == 'enabled' else '[--]'
            print(f"{status} {svc['name']}")
            if 'loadBalancer' in svc:
                for server in svc['loadBalancer'].get('servers', []):
                    print(f"     -> {server.get('url', 'N/A')}")
        
        student_svcs = [s for s in services if 'student' in s['name'].lower()]
        if student_svcs:
            print(f'\n[OK] {len(student_svcs)} student service(s)')
            return True
        print('\n[FAIL] No student services')
        return False
    except Exception as e:
        print(f'[FAIL] {e}')
        return False

test_traefik_services()

## Layer 6: Docker Network Connectivity

In [ ]:
def test_docker_network():
    try:
        result = subprocess.run(
            ['docker', 'network', 'inspect', 'hydra_students_net'],
            capture_output=True, text=True
        )
        if result.returncode != 0:
            print('[FAIL] Network hydra_students_net not found')
            return False
        
        network = json.loads(result.stdout)[0]
        containers = network.get('Containers', {})
        
        print('Containers on hydra_students_net:')
        print('-' * 50)
        for cid, info in containers.items():
            print(f"  {info.get('Name', cid)}: {info.get('IPv4Address', 'N/A')}")
        
        names = [c.get('Name', '') for c in containers.values()]
        has_traefik = any('traefik' in n.lower() for n in names)
        has_student = any('student' in n.lower() for n in names)
        
        if has_traefik and has_student:
            print('\n[OK] Traefik and student containers connected')
            return True
        if not has_traefik:
            print('\n[FAIL] Traefik not on network')
        if not has_student:
            print('\n[FAIL] No student containers on network')
        return False
    except Exception as e:
        print(f'[FAIL] {e}')
        return False

test_docker_network()

## Layer 7: Direct Container Service Test

In [ ]:
def test_container_direct():
    result = subprocess.run(
        ['docker', 'inspect', '-f', '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}', f'student-{TEST_USER}'],
        capture_output=True, text=True
    )
    if result.returncode != 0:
        print(f'[FAIL] Container student-{TEST_USER} not found')
        return False
    
    ip = result.stdout.strip()
    if not ip:
        print('[FAIL] Container has no IP')
        return False
    
    print(f'Container IP: {ip}')
    success = True
    
    print('\ncode-server (8443):')
    try:
        r = requests.get(f'http://{ip}:8443/', timeout=5)
        print(f'  [OK] status={r.status_code}')
    except Exception as e:
        print(f'  [FAIL] {e}')
        success = False
    
    print('\nJupyter (8888):')
    try:
        r = requests.get(f'http://{ip}:8888/', timeout=5)
        print(f'  [OK] status={r.status_code}')
    except Exception as e:
        print(f'  [FAIL] {e}')
        success = False
    
    return success

test_container_direct()

## Layer 8: Full Route Test (Through Traefik)

In [ ]:
def test_full_route():
    tests = [
        ('Dashboard', f'http://{HYDRA_HOST}/dashboard'),
        ('VS Code', f'http://{HYDRA_HOST}/students/{TEST_USER}/vscode/'),
        ('Jupyter', f'http://{HYDRA_HOST}/students/{TEST_USER}/jupyter/'),
    ]
    
    print('Route tests through Traefik:')
    print('-' * 50)
    
    all_ok = True
    for name, url in tests:
        try:
            r = requests.get(url, timeout=10, allow_redirects=False)
            code = r.status_code
            if code == 200:
                result = '[OK]'
            elif code == 302:
                result = f'[REDIRECT] -> {r.headers.get("Location", "?")}'
            elif code == 401:
                result = '[AUTH] forwardAuth working'
            elif code == 404:
                result = '[FAIL] not found'
                all_ok = False
            elif code == 502:
                result = '[FAIL] bad gateway'
                all_ok = False
            else:
                result = f'[{code}]'
            print(f'{name}: {result}')
        except Exception as e:
            print(f'{name}: [FAIL] {e}')
            all_ok = False
    
    return all_ok

test_full_route()

## Layer 9: ForwardAuth Flow Test

In [ ]:
def test_forward_auth():
    auth_url = 'http://localhost:6969/auth/verify'
    print('ForwardAuth test:')
    print('-' * 50)
    try:
        r = requests.get(auth_url, timeout=5)
        print(f'No session: {r.status_code}')
        if r.status_code == 401:
            print('[OK] Rejecting unauthenticated')
            return True
        elif r.status_code == 200:
            print('[WARN] Passing without session (dev mode?)')
            return True
        print(f'[WARN] Unexpected status')
        return False
    except Exception as e:
        print(f'[FAIL] {e}')
        return False

test_forward_auth()

## Summary: Run All Tests

In [ ]:
def run_all_tests():
    results = {
        'DNS': test_dns(),
        'Traefik Entrypoint': test_traefik_entrypoint(),
        'Routers': test_traefik_routers(),
        'Middlewares': test_traefik_middlewares(),
        'Services': test_traefik_services(),
        'Docker Network': test_docker_network(),
        'Container Direct': test_container_direct(),
        'Full Route': test_full_route(),
        'ForwardAuth': test_forward_auth(),
    }
    
    print('\n' + '=' * 50)
    print('SUMMARY')
    print('=' * 50)
    for test, passed in results.items():
        print(f"{'[OK]' if passed else '[FAIL]'} {test}")
    
    failed = sum(1 for v in results.values() if not v)
    if failed:
        print(f'\n{failed} test(s) failed')
    else:
        print('\nAll tests passed')

# run_all_tests()